# Spatialize

### Add the src folder to the path

In [ ]:
#! yes | pip3 install -U umap-learn

In [ ]:
#!pip install scikit-learn

In [ ]:
#! pip uninstall h2o4gpu

In [1]:
import sys
import os

root_path = os.path.dirname(os.getcwd())
src_path = os.path.join(root_path, "src")
sys.path.insert(0, src_path)

In [2]:
%load_ext autoreload
%autoreload 2

## Common imports

In [3]:
%matplotlib inline
from matplotlib import pyplot as plt
import torch
import numpy
from tissue_purifier.plot_utils.plot_results import plot_all_maps, plot_embeddings
from tissue_purifier.misc_utils.misc import get_z_score

# Make a spatialize

In [4]:
from tissue_purifier.data_utils.datamodule import SlideSeqTestisDM
from tissue_purifier.model_utils.dino import DinoModel

config_dict_default_dm = SlideSeqTestisDM.get_default_params()
dm = SlideSeqTestisDM(**config_dict_default_dm)

#config_dict_default_dino = DinoModel.get_default_params()
#dino = DinoModel.load_from_checkpoint(checkpoint_path=ckpt_file)

-----> running datamodule init


In [5]:
dm.prepare_data()

-----> running datamodule prepare_data
number of elements ---> 31659
The dense shape of the image is -> torch.Size([9, 1168, 1168])
number of elements ---> 33059
The dense shape of the image is -> torch.Size([9, 1170, 845])
number of elements ---> 39206
The dense shape of the image is -> torch.Size([9, 1169, 1170])
number of elements ---> 27194
The dense shape of the image is -> torch.Size([9, 1166, 1170])
number of elements ---> 42776
The dense shape of the image is -> torch.Size([9, 1170, 1170])
number of elements ---> 33441
The dense shape of the image is -> torch.Size([9, 1154, 1155])


In [6]:
dm.setup(stage=None)

In [7]:
train_loader = dm.train_dataloader()
sparse_images = train_loader.dataset.imgs

In [8]:
sp_img = sparse_images[0]

In [9]:
vars(sp_img)

{'x_name': 'x',
 'y_name': 'y',
 'classifier_classes_': None,
 'classifier_counts_': None,
 'original_data':        Unnamed: 0         barcode            x            y   cell_type
 0               0  ACCTTTTTTTTTTT  2911.083333  3180.472222          ES
 1               1  CGGGATTTTTTTTT  3037.054795  4285.363014          RS
 2               2  CATTCATTTTTTTT  1664.345238  3552.726190         SPC
 3               3  GGCTACTTTTTTTT  4048.823204  2583.165746       Myoid
 4               4  CTTGATGTTTTTTT  3072.972222  2726.277778         SPC
 ...           ...             ...          ...          ...         ...
 31654       31654  GTAGAAAAAAAAAA  3848.592593  2712.548148  Macrophage
 31655       31655  GGACAAAAGAAACA  3752.410959  2630.520548          ES
 31656       31656  CGCAGTTTCAAAAC  1795.675325  1678.961039         SPC
 31657       31657  GCGCTCAAAAAACA  4204.135135  3657.770270         SPC
 31658       31658  AAACAAAAAAAAAA  4145.626761  3055.626761     Sertoli
 
 [31659 rows x

In [10]:
mask_file = 'classifier_spatialize.pt'
classifier_classes_, classifier_counts_ = torch.load(mask_file, map_location=torch.device('cpu'))
sp_img.classifier_classes_ = classifier_classes_
sp_img.classifier_counts_ = classifier_counts_

In [11]:
sp_img.shape

torch.Size([9, 1168, 1168])

In [14]:
sp_img.classifier_probs_.shape

AttributeError: 'SparseImage' object has no attribute 'classifier_probs_'

In [15]:
sp_img.add_probs_to_original_data()

After interpolation (5, 31659)


In [16]:
vars(sp_img)

{'x_name': 'x',
 'y_name': 'y',
 'classifier_classes_': array([0, 1, 2, 3, 4]),
 'classifier_counts_': tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
    

In [17]:
df = sp_img.export_original_data()

In [18]:
df

,Unnamed: 0,barcode,x,y,cell_type,classifier_probs_0,classifier_probs_1,classifier_probs_2,classifier_probs_3,classifier_probs_4
0,0,ACCTTTTTTTTTTT,2911.083333,3180.472222,ES,0.160956,0.820489,0.001479,0.016908,0.000168
1,1,CGGGATTTTTTTTT,3037.054795,4285.363014,RS,0.257043,0.713814,0.000365,0.028612,0.000166
2,2,CATTCATTTTTTTT,1664.345238,3552.726190,SPC,0.105627,0.894228,0.000101,0.000021,0.000023
3,3,GGCTACTTTTTTTT,4048.823204,2583.165746,Myoid,0.234709,0.728571,0.000080,0.036501,0.000139
4,4,CTTGATGTTTTTTT,3072.972222,2726.277778,SPC,0.059922,0.926853,0.000774,0.012286,0.000165
...,...,...,...,...,...,...,...,...,...,...
31654,31654,GTAGAAAAAAAAAA,3848.592593,2712.548148,Macrophage,0.251077,0.703105,0.000122,0.045517,0.000179
31655,31655,GGACAAAAGAAACA,3752.410959,2630.520548,ES,0.204907,0.764899,0.000119,0.029900,0.000176
31656,31656,CGCAGTTTCAAAAC,1795.675325,1678.961039,SPC,0.237040,0.762841,0.000030,0.000039,0.000051
31657,31657,GCGCTCAAAAAACA,4204.135135,3657.770270,SPC,0.321699,0.633275,0.000428,0.044160,0.000438
